# Chunking Guidance for Retireval Augmentation Generation 

Retrieval Augementation and Generation (RAG) systems are increasingly popular as a way to bootstrap an LLM on your own information. 

Generally it is easy to implement a proof of concept that passes the infamous "vibe" check fairly quickly, whilst this might get an engineer 70-80% of the way, as with any data driven system, the devil really is in the details when it comes to squeezing out the extra performance that gives end users confidence.

Through this notebook we'll look at how to best understand a corpus of documents, some analysis that can help determine some of the parameters of your chunking, and how to go about running experiments to see what really works for your data. We'll run the baseline experiment "from first principles" i.e. we won't use any fancy tooling that abstracts what's going on under the hood. For subsequent experiments, we'll use separate notebooks and show how some popular tooling can make life a lot easier!

Naturally, RAG systems have a lot of components, where possible we'll introduce core concepts, but we won't deep dive on everything here.

## Purpose

This notebook aims to give direction to software engineers in how they might approach evaluating, and selecting a chunking methodology for a RAG system. It is not meant to be an accelerator or template. There are other great resources for that like the [RAG Experiment Accelerator](https://github.com/microsoft/rag-experiment-accelerator). Instead this is meant to be complementary to those assets, highlight the thinking and decisions that you might make whilst approaching a chunking problem.

This notebook does NOT intend on covering end to end evaluation of RAG systems, as that is a much broader topic that dives into information retrieval. Resources for tuning those elements of a RAG system can be found [here]().



## Outline

1. [Exploring your data](#exploring-your-data)
2. [Chunks, Words, Tokens and Contexts](#chunks-words-token-and-contexts)
3. [Experiment Setup](#experiment-setup)
4. [Building an Evaluation Dataset](#building-an-evaluation-dataset)
5. [RAGE-1: RAG Experiment Harness](#rage-1-rag-experiment-harness)
6. [RAGE-2: RAG Evaluation Harness](#rage-2-rag-evaluation-harness)

Let's get started!

## Loading a Corpus

First things first, let's load some text data to work with. Let's go with the [pubmed summarisation dataset](https://huggingface.co/datasets/ccdv/pubmed-summarization). We'll download from hugging face, but for simplicity we'll convert the dataset to pandas, which most data pro's are familiar with. To make sure no information is lost, we'll do a quick record check count and verify that the descriptive statistics are equal for the aticle lengths.

In [ ]:
# Imports

from datasets import load_dataset
from uuid import uuid4
from pprint import pprint
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import tiktoken as tk
import random
import json
from multiprocessing import Pool

# Set to pubmed or arxiv
publication = 'pubmed'

dataset = load_dataset(f'scientific_papers',publication,split='train', trust_remote_code=True)

# Convert to a pandas dataframe and do some housekeeping
ds = dataset.to_pandas()
ds['doc_id'] = [str(uuid4()) for _ in range(len(ds))]
ds['article_len'] = ds['article'].apply(lambda x: len(x.split()))

ds.drop(columns=['abstract'],inplace=True)

# Take a look at our data
display(ds.head())

# Check to make sure that the conversion hasn't caused any issues

# Number of records
print("Number of records in original dataset: ", len(dataset))
print("Number of records in transformed dataset: ", ds.shape[0])

# Descriptive statistics for article length
original_lengths = [len(article.split()) for article in dataset['article']]

if pd.Series(original_lengths).describe().equals(ds['article_len'].describe()):
    print("\nArticle length statistics match!")

    print("\nDescriptive statistics for article length in original dataset:")
    display(pd.Series(original_lengths).describe())

    print("\nDescriptive statistics for article length in transformed dataset:")
    display(ds['article_len'].describe())

else:
    print("\nArticle length statistics do not match! Please check the conversion.")

## Exploring your data

One of the main things that we are concerned with when building RAG systems is the length of the data that we intend to feed in to an LLM. Even with context windows (i.e. the length of the input + output to/from an LLM) are becoming increasingly large, but we still don't really know if simply throwing more data at an LLM effective, and it is definitely not quick.

Let's start by understanding just how long a typical article is, and get a feel for the boundaries and overall distribution of our data.

An easy way to do this is to look at some basic stats, and draw a histogram.

In [ ]:
display(ds.describe())

fig = plt.figure(figsize=(10, 6))
sns.histplot(ds['article_len'], bins=50, kde=True)
plt.show()

Both the table and the diagram show what's called a heavy right skew - meaning that there are a small number of extremely large values. 

What we do with these extra long articles is an important decision. For some data problems you'd look to exclude outliers. But given this is an information retrieval problem, we'd like to keep as much data as possible. 

What we need to do, is understand  *why* some of these articles are so long, and deal with them accordingly

First, lets zoom in on the distribution and if we can find a more reasonable cut off point.

In [ ]:
ds.describe(percentiles=[0.75,0.8, 0.9,0.95, 0.99])

The problem is definitely in the high end of town, with a jump from 10k to 112k in number of words for the last percentile. Let's take a closer look at the raw data and see if we can tell what's going on.

In [ ]:
# Get the indices of the articles with the longest length
longest_articles = ds['article_len'].nlargest(5).index
for idx in longest_articles:
    print(f'Article Length: {ds["article_len"][idx]}\n')
    print(ds['article'][idx]+'\n')
    print('\n')

At first glance, it appears that the two main causes of long documents are either:
- LaTeX package inclusions (mathematical formatting for scientific documents)
- Data tables (from pharmaceutical research by the look of it)

In practice we would want to spend more time understanding the drivers behind the outliers, and address as many as possible. 

However, for the purposes of this exercise we will focus on the LaTeX issue. 

Data tables could be solved through an application of difference pdf cracking techniques (e.g. [Azure Document Intelligence](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/concept-retrieval-augmented-generation?view=doc-intel-4.0.0)), but that is out of scope for this notebook.

Let's remove the the lines which include LaTeX and take another look at the adjusted distributions.

In [ ]:
from helper.general import remove_latex_packages

# Remove LaTeX package inclusions from the articles
ds['article'] = ds['article'].apply(remove_latex_packages)

# Recalculate article lengths
ds['article_len'] = ds['article'].apply(lambda x: len(x.split()))

display(ds.describe(percentiles=[0.75,0.8, 0.9,0.95, 0.99]))

This has made a difference, but 95k is still very large. For now, let's exclude the longer documents, storing them in another dataframe for analysis later.

Once we remove the odd docs, we'll check our distribution again to make sure that we now have something workable.

In [ ]:
from helper.general import remove_over_percentile

#apply helper function from utils module
ds_99pct, ds_outliers = remove_over_percentile(ds, 'article_len', .99)

fig = plt.figure(figsize=(10, 6))

# Plot the first histogram on the first subplot
sns.histplot(ds_99pct['article_len'], bins=100, kde=True)
plt.title('Article Length')


# Display the plots
plt.show()

display(ds_99pct.describe())

Let's take a quick look at the smallest articles too. The above histogram indicates that there are a number of articles that have length of less than 500 words. Let's check formally and then take a peek.

In [ ]:
count_tiny_articles = ds_99pct['article_len'][ds_99pct['article_len'] < 500].count()

print(f'Number of articles with less than 500 words: {count_tiny_articles}')

smallest_articles = ds_99pct['article_len'].nsmallest(count_tiny_articles).index
for idx in smallest_articles:
    print(f'Article Length: {ds_99pct["article_len"][idx]}\n')
    print(ds_99pct['article'][idx]+'\n')
    print('\n')

Scanning through the articles there are some empty records, but the majority seem to be fractions of documents rather than complete articles. We can be confident that this isn't an issue with the hugging face dataset conversion as we tested that at the start of the notebook. We can reasonably conclude that these are invalid records and exclude them from our system. It's worth calling out that this exclusion is different from excluding the large ones, as the are malformed at source and there is little that we can do to remediate without calling a friend!

In [ ]:
ds_clean = ds_99pct[ds_99pct['article_len'] >= 500]

Now that we have our cleansed dataset, let's start talking about tokens, why it's good to know about them, and how they relate to designing a RAG system.

## Chunks, Words, Token and Contexts

Despite feeling like LLMs converse in our language, there's a few things that go in behind the scenes that translate our verbiage into something an algorithm understands. 

Firstly, the text is `tokenized`, which means words are split into a list of `tokens`. Think of this a bit like stemming in NLP. For shorter words, the ratio of tokens to words can be 1:1 (i.e. the word = the token), but for longer, or more complex words the ratio can be far higher. 

These lists are then mapped into numerical vectors that the algorithm can understand. For a given corpus, we could work out the exact ratio - in fact, let's do that!

(for a visual explanation of tokens see [this](https://www.tokencounter.io/) excellent resource)

First we will apply the appropriate encoding to the text, then we simply sum up the number of tokens and words in the corpus and calculate the number of tokens / word.

In [ ]:
# run the tokeniser over the articles and store the results in the DataFrame
# Given the size of the dataet this can take up to 10 minutes Let's select a 
# random sample of 5000 articles to reduce the time down to ~ 10 seconds
# Feel free to grab a beverage and run the analsis on the entire dataset to
# compare the results if you have the time!

subset = True

if subset == True:
    random.seed(42)
    random_doc_ids = random.sample(list(ds_clean['doc_id'].unique()), 5000)

    # Subset the DataFrames
    ds_clean = ds_clean[ds_clean['doc_id'].isin(random_doc_ids)]

encoding = tk.encoding_for_model('gpt-3.5-turbo')

article_tokens = ds_clean['article'].apply(encoding.encode)

# check if columns already exist
if 'article_tokens' in ds_clean.columns:
    ds_token_count_subset = ds_clean.drop(columns=['article_tokens'])

ds_clean = ds_clean.assign(article_tokens=article_tokens)
ds_clean['article_tk_len'] = ds_clean['article_tokens'].apply(lambda x: len(x))
ds_clean['token_ratio'] = ds_clean['article_tk_len']/ds_clean['article_len']

total_words = ds_clean['article_len'].sum()
total_tokens = ds_clean['article_tk_len'].sum()

ratio = total_tokens/total_words

print(f'The ratio of tokens to words is {ratio:.2f}')
print(f"The largest article has {ds_clean['article_tk_len'].max()} tokens")
print(f"The smallest article has {ds_clean['article_tk_len'].min()} tokens")
print(f"The mean number of tokens is : {ds_clean['article_tk_len'].mean()}")

Now we know the overall ratio we can make some assumptions around cost and how much information we feed the generation step as context.

Before we move on, let's just take a look at the distribution of ratios - some articles might use more complex language than others.

For this we'll use a violin plot which is similar to a box plot, but can be used to give great visuals across multiple classes (e.g. pharma vs biotech articles).

The plot below shows that we have a few articles that have high token ratios - an interesting problem for another time!

In [ ]:
# create a violin plot of token ratios
fig = plt.figure(figsize=(10, 6))
sns.violinplot(x='token_ratio', data=ds_clean)
plt.title('Token Ratio')
plt.show()

# TODO: Wouldn't it be fun to find two classes of articles with different token ratios and compare them?


### Why do we care about this?

How many records do we want to include in our Augmentation step when constructing the generation prompt? Say we're using GPT-35-Turbo, we have aprx 4000 tokens to play with. This is both **input and output**.

Let's assume the following:

1. We have a prompt template which is a total of 500 tokens, including our guardrails, instructions and any other boiler plate commentary that needs to be input to the generation step.
2. We allow for up to 500 tokens in a response. 

$$Guardrails + Instructions + Estimated Response Length + Retrieved Context < Model Token Limit$$


Meaning with **GPT-35-Turbo**:
$$ 500 + 500 + Retrieved Context < 4000 $$

or $ Retrieved Context < 3000 $

This leaves us with 3000 tokens to play with. If we assume a chunk size  of 500, that gives us 6 records in our retrieval step. In fact, this might be a good starting point. It glosses over whether or not 500 tokens is enough to capture something that is semantically relevant and (more importantly!) useful for the generation step, but we'll take a look at that as we experiment with different chunking strategies.

Now we have some idea of a driving non functional contraint. Let's talk about what makes a good chunk.

### What makes a good chunk?

Let's start by really addressing what a chunk is intended to do. It's sole purpose is to provide context to a model that enhances the available "knowledge" available to the LLM to better answer a given question.

To do that well, a chunk should:

1. Be relevant to the query or task
2. Be factually accurate and avoid misleading information
3. Be specific - focused information rather than overly broad content
4. Concise and information dense. Conveying information effectively within a limited space
5. Referenceable. A user should always be able to go back and see the chunk in context of it's parent document

Let's talk about each in turn:

#### Relevance

This is primarily the focus of optimising search results. What metrics do you choose to select you chunks when executing the retrieval step. [Cosine Similarity]() and the [dot product]() are two commonly used metrics. You may expand this by performing a hybrid search (adding metadata to your quesry - for example published dates, authors etc.), or even looking at strategies for [re-ranking](). The specifics of these techniques are beyond the scope of this notebook as they fall into a separate, and expansive discipline (Maybe another notebook!). In our example we are simply taking the top n records based on cosine similarity.

#### Factual Accuracy

This is fundamentally a data governance problem. Where did your data come from, is it up to date, can you trust the source. An example provided in a recent epiisode of [Practical AI]() gave the example of an HR bot being queried on annual leave policies. One data source could be the official HR policy which contains relevant, up to date information, whereas another source could be chat logs that are dated and included discussions from other georgaphies that aren't relevant to the users context. In our example, we are essentially outsourcing the governance problem by trusting the Hugging Face dataset and PubMed as sources.

#### Specificity

This is where our chunking strategy starts to play a part. How many topics does a chunk cover? Is it general in nature, or does it cut to the point. It may be that we find that chunk length correlates well with specificity. Perhaps we need ot look at topics, or semantic consistency in chunks. More on all of this to come!

#### Concise

We've illustrated the finite number of tokens we have to play with in the previous section. We ideally want information dense chunks that provide the correct context without rambling and consuming too many tokens

#### Referencable

Whilst the ultimate answer is generated by an LLM, often users will want to check the source. It is best practice to provide both the context, and the source of the context items back to the user for verification and / or further research. We'll achieve this by making use of metadata in our vector index.

Now we understand our data, how we need to manage the inputs to a RAG system, and what a good output looks like. Let's set up our experiments.


## Experiment Setup

At Microsoft, we use a framework called the [AI Garden](https://github.com/cse-labs/ai-garden/tree/main). This allows us to group related experiments into "experiment families" and collaborate across groups and engagements in sharing knowledge and learnings.

In this repo we have the [experiments](experiments) directory which contains our expermiment family and distinct experiment plans and results. We also include Architectural Decision Records (ADRs) which include important technical decisions that impact the outcome, along with reasoning. I encourage you to check these out! 

### Subset data

Given this is an illustration of the process and thinking, now that we have a clean dataset we're going to go one step further an subset the data down to 50 randomly selected articles - mostly because I'm terrified of my GPT4 bill! 

In [ ]:
# select a random 50 unique doc_ids and subset ds_cleam - note we will use this again later!
random.seed(42)
random_doc_ids = random.sample(list(ds_clean['doc_id'].unique()), 50)

# Subset the DataFrames
ds_subset = ds_clean[ds_clean['doc_id'].isin(random_doc_ids)]

#write out the dataset to a csv file
ds_subset.to_csv('data/ds_subset.csv', index=False)

### Building an Evaluation Dataset

Now that we have the data, let's think about how we are going to evaluate our chunking strategies. The evaluation metrics are outlined in the [experiment documentation](./experiments/00-chunking-strategies-family.md). The foundation of evaluation is a good set of question / answer pairs.

Why do we do this?

#### Simulating Real-World Use:

- QA pairs mimic how users interact with a RAG system. Users ask questions, and the system retrieves documents and generates answers based on those documents.
- By testing with good QA pairs, you ensure the RAG system performs well in scenarios it's designed for.

#### Evaluating Retrieval Accuracy:

- Good QA pairs often have answers that can be found within a relevant document. This allows you to see if the RAG system retrieves the documents that actually hold the answer.

- Poor retrieval throws off the entire RAG process, so evaluating retrieval accuracy is crucial.

#### Assessing Answer Quality:

- QA pairs with well-defined answers provide a benchmark for the RAG system's generation capabilities

- You can compare the generated answer to the actual answer in the retrieved document to see if the RAG system effectively uses the information.

#### Identifying System Biases:

- A diverse set of QA pairs helps identify potential biases in the RAG system.

- For instance, if the system struggles with specific question types or topics, the QA pairs will reveal these weaknesses.

#### Not just any QA pairs will do:

- Good QA pairs should be well-formed, grammatically correct, and cover a range of difficulty levels and topics relevant to your RAG system's intended use.

- In essence, good QA pairs provide a realistic testing ground to assess how well your RAG system retrieves information and generates accurate and relevant answers.

Now, I don't have the time or expertise to generate a set of Q&A for these pubmed articles. A popular approach is to use a leading LLM to read the documents and create the question answer pairs for you. Ideally this (or at least a subset) would be reviewed by a selection of subject matter experts before being used as a the basis for measurement.

To do this, we will create a prompt template that submits each article, and generates a set of 5 question answer pairs in a specific format. Once we have the tmeplate, we can submit the prompts to our model of choice. 

>Note: Given this requires a structured output, we recommend using gpt-4 for this step. There are other ways to apply structure to outputs, but for simplicity's sake we'll use a the sledgehammer.

In [ ]:
from helper.openai_utils import general_prompt, create_client
from helper.general import convert_to_dict
from rag.data_prep import generate_qa_prompt

# Create an OpenAI client
oai_client = create_client()
model = os.getenv("QA_MODEL")

def _process_article(article):
    return general_prompt(oai_client, generate_qa_prompt(article), model=model)


def create_qa_pairs(client, model, articles, save_output=True, parallel=True):
    if parallel is True:
        with Pool() as pool:
            results = pool.map(_process_article, articles)

    else:
        prompts = [generate_qa_prompt(article) for article in ds_subset["article"]]
        results = [general_prompt(client, prompt, model=model) for prompt in prompts]

    # Save the results to a file
    qa_pairs = convert_to_dict(results)

    # TODO: Make sure this is in the format expected by AI Studio and clean up csv
    if save_output is True:
        with open("data/qa_pairs.jsonl", "w") as f:
            json.dump(qa_pairs, f, indent=4)

    questions = [pair["question"] for pair in qa_pairs]
    answers = [pair["answer"] for pair in qa_pairs]

    # Create a DataFrame with the questions and answers
    qa_df = pd.DataFrame({"question": questions, "answer": answers})

    return qa_df

# if the data is already saved, load it
if os.path.exists('data/qa_pairs.csv'):
    qa_df = pd.read_csv('data/qa_pairs.csv')
else:
    qa_df = create_qa_pairs(oai_client, model, ds_subset["article"], save_output=True, parallel=True)

## RAGE-1: RAG Experiment Harness

### Setting up the Retrieval step

What do we need to do next. Given a dataframe containing our chunked documents we must:

- Chunk the documents
- Embed the chunks.
- Store it in a vector database.
- Query the db using our ground truth (GT) questions.
- Generate a final response.
- Store the Question, Answer, GT Answer, Context for our evaluation framework.

Let's follow our [baseline experiment](experiments/01-chunking-strategies-baseline.md) as an example. Here we create chunks with a fixed length (in words), and an overlap (also in words). This is intentionally naiive to provide a clear baseline to measure the relative improvements of subsequent experiments.

#### Chunking the documents

Based on our earlier analysis, let's take a chunk size of 400 words with an overlap of 50. We can calculate the exact length of requests later, but 400 words is approximately 500 tokens per chunk based on our analysis and we should be able to fit multiple results into our context.


In [ ]:
from rag.chunking import chunk_string_with_overlap

# Create a new DataFrame with each chunk as a separate row
chunks = []
doc_ids = []
chunk_ids = []
for idx, row in ds_subset.iterrows():
    article_chunks = chunk_string_with_overlap(input_text=row['article'], chunk_length=400, overlap=50)
    chunks.extend(article_chunks)
    doc_ids.extend([row['doc_id']] * len(article_chunks))
    chunk_ids.extend([f"{row['doc_id']}-{i+1}" for i in range(len(article_chunks))])

ds_chunked = pd.DataFrame({'doc_id': doc_ids, 'chunk_id': chunk_ids, 'chunks': chunks})
ds_chunked.to_csv('data/ds_chunked.csv', index=False)

# Worl out the average number of chunks per document
avg_chunks_per_doc = ds_chunked.groupby('doc_id').size().mean()
print(f'Average number of chunks per document: {avg_chunks_per_doc}')

#### Embed the chunks

We will use the ada-v2 embedding model for this example as it is fairly powerful and generally understood. It's worth noting that this will not always be the best model, paticularly when data contains topics and content that relate to finding outside of the embedding model's training data. Fine tuning an embedding model on a specific corpus (particularly in the case of highly specialised data) is also a popular option.

Most vector databases implement a wrapper around common embedding functions. Here we will configure and use the wrapper for openai embeddings in ChromaDB. This function is used to embed all documents in a collection, and also to embed queries as they come in.

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())   
# Specify Embedding model
embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")

# Used in chromadb to embed docs and queries
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_type="azure",
                api_version=os.getenv("OPENAI_API_VERSION"),
                model_name=embedding_model
            )
# Create a new collection in ChromaDB

from chromadb import PersistentClient

chroma_client = PersistentClient(path="./data/chroma_db")

index_name = "baseline_pubmed_articles"
collection = chroma_client.get_or_create_collection(name=index_name,embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})

collection.add(
    # embeddings=ds_chunked['ada_v2'].tolist(),
    documents=ds_chunked['chunks'].tolist(),
    metadatas=[{"doc_id": doc_id} for doc_id in ds_chunked['doc_id']],
    ids=ds_chunked['chunk_id'].tolist()
    )
    

Now let's take a quick look at an example question and the response.

In [ ]:
results = collection.query(
    query_texts=[qa_df['question'][3]],
    n_results=5
)
pprint(qa_df['question'][3])
pprint(results)


We can see that the query returns the ID's, scores (distances), metadata and documents (chunks) for the top 5 documents in the collection when scored by cosine similarity. The chunks will form our context, and we will use the metadata for lineage. In our case, we see that there are a number of chunks from the same document - this can be seen as a positive indicator given 

Now let's take a look at our augmentation and generation steps and apply this at scale!

#### Augmentation and Generation

Here we are enriching the question with the new (and hopefully relevant!) context we have unearthed from the vector database. To do that, we'll need another prompt template. Once we have this, we can submit the prompt to our generation model and receive the answer to our question.

Let's use the previous example

In [ ]:
from rag.augmentation import get_context, contruct_prompt

context = get_context(qa_df['question'][3], collection, 3)
prompt = contruct_prompt(context, qa_df['question'][3])

pprint(prompt)

# Count the number of tokens in the prompt
prompt_tokens = encoding.encode(prompt)
print(f"Number of tokens in the prompt: {len(prompt_tokens)}")

We can see that we've created a new prompt that's ready to be submited to our generation model. Let's take a look at a single call.

In [ ]:
response = general_prompt(oai_client, prompt, model=os.getenv("GEN_STEP_MODEL"))

print(f"Question: {qa_df['question'][3]}")
print(f"Correct Answer: {qa_df['answer'][3]}")

print(f"Generated Answer: {response}")

This is looking pretty good! What you've likely just experienced is the infamous "vibe check" for LLM based applications. We'll get on to more formal measurement soon. But first, let's get ansers to all 250 questions.

In [ ]:
model = 'gpt-4' #os.getenv("GEN_STEP_MODEL")
multi_threading = True

#Create a distinct copy of qa_df to store the results
results_df = qa_df.copy()

if os.path.exists(f'data/results-{index_name}-{model}.csv'):
    print("File exists, reading in...")

    results_df = pd.read_csv(f'data/results-{index_name}-{model}.csv')

else:
    def generation_step(question):
        context = get_context(question, collection,3)
        prompt = contruct_prompt(context, question)
        return general_prompt(oai_client, prompt, model=model)

    if multi_threading == True:
        with Pool() as pool:
            results_multiprocessing = pool.map(generation_step, results_df['question'])
        results_df['response'] = results_multiprocessing

    else:
        results_df['response'] = results_df['question'].apply(lambda x: generation_step(x))

    #TODO: Refactor this so only one call for context
    # Check if the column exists
    if 'context' not in qa_df.columns:
        results_df['context'] = [get_context(q, collection) for q in results_df['question']]

    #write out to CSV
    results_df.to_csv(f'data/results-{index_name}-{model}.csv', index=False)

display(results_df.head())

Now we have a dataframe with the questions, true answers, generated answers, and the context used to generate them, we can start to look at whether or not the answers are any good. To do that, we'll use the evaluation framework in Azure AI Studio.

## RAGE-2: RAG Evaluation Harness